In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [4]:
data = loadmat('ex3data1.mat')

In [6]:
type(data)


dict

In [7]:
data


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [10]:
data['X'].shape, data['y'].shape


((5000, 400), (5000, 1))

In [24]:
def sigmoid(z):
    return 1 /(1 + np.exp(-z))

In [98]:
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply((-y), np.log(sigmoid(X * theta.T)))
    second = np.multiply((1-y), np.log(1-sigmoid(X * theta.T)))
    reg = (learningRate) / (2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    
    return np.sum(first - second) / len(X) + reg
    # 因为我们未对 进行正则化，所以梯度下降算法将分两种情形：
    # 为什么啊，我没看懂呢。


In [31]:
def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    

    for i in range(parameters):
        term = np.multipy(error, X[:, i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = np.sum(term) / len(X) + (learningRate / len(X)) * theta[:,i]

    return grad
    

In [102]:
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    # grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
        
        
    # intercept gradient is not regularized
    grad[0,0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    
    return np.array(grad).ravel()


    
    

In [96]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

In [53]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

In [55]:
rows, params

(5000, 400)

In [57]:
X = np.insert(data['X'], 0,values = np.ones(rows),axis=1)

In [58]:
X.shape


(5000, 401)

In [60]:
theta = np.zeros(params+1)

In [72]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

In [73]:
X.shape, y_0.shape, theta.shape, all_theta.shape

NameError: name 'all_theta' is not defined

In [74]:
all_theta = np.zeros((10, params + 1))

In [76]:
all_theta.shape

(10, 401)

In [77]:
X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [80]:
np.unique(data['y'])


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [103]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)

In [104]:
all_theta


array([[-2.38226925e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30432887e-03, -7.25106432e-10,  0.00000000e+00],
       [-3.18468826e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46368208e-03, -5.08909111e-04,  0.00000000e+00],
       [-4.79563242e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86332926e-05, -2.47096397e-07,  0.00000000e+00],
       ...,
       [-7.98719598e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.96022489e-05,  7.22582523e-06,  0.00000000e+00],
       [-4.57015294e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33613523e-03,  9.99418923e-05,  0.00000000e+00],
       [-5.40393035e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16569695e-04,  7.86977435e-06,  0.00000000e+00]])

In [107]:
def predict_all(X, all_theta):
    rows=X.shape[0]
    params= X.shape[1]
    num_labels= all_theta.shape[0]
    
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    h = sigmoid(X * all_theta.T)
    
    h_argmax = np.argmax(h, axis=1)
    
    h_argmax = h_argmax + 1
    
    return h_argmax
    

In [108]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.46%


In [114]:
data['X'].shape

(5000, 400)

In [111]:
all_theta.shape


(10, 401)